<a href="https://colab.research.google.com/github/tarushi1/CSI_MLTask/blob/main/foodpoisoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
#Loading neccessary libraries
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import os
from sklearn.cluster import KMeans, SpectralClustering
import seaborn as sns
import sys
from time import time

In [86]:
#Connecting drive and colab
from google.colab import drive
drive.mount('/content/drive')
rev= pd.read_csv("/content/drive/MyDrive/CSI_MLTask/McDonalds-Yelp-Sentiment-DFE.csv")
rev.sample(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Number,review
56,67984622,My wife and I were at the McDonalds on 12109 N...
33,67984599,I have stayed away from most fast food places ...
65,67984631,This place is really bad. Stay away at all cos...
59,67984625,The reason I gave this McDonald's 2 stars inst...
19,67984585,The sweet tea and fries are always horrible bu...


In [87]:
#dropping the missing/NA values
rev.review.isna().sum()
rev.review[(rev.review.str.len() == 0) | (rev.review == "")].shape[0]
print("Dimensions of dataset before dropping the NAs:",rev.shape)
rev = rev.dropna(subset=['review'])
print("Dimensions of dataset after dropping the NAs:",rev.shape)

Dimensions of dataset before dropping the NAs: (112, 2)
Dimensions of dataset after dropping the NAs: (103, 2)


In [88]:
#tokenization
vectorizer = CountVectorizer(stop_words='english')
Vs= vectorizer.fit_transform(rev.review.values).toarray()

In [89]:
#TF IDF transformation
def idf_transform(word_col):
    w = len(word_col[np.nonzero(word_col)])
    return np.log(len(word_col)/(w + 1))

def tf_idf(bow):
    tf = np.log(bow + 1)
    idf = np.apply_along_axis(idf_transform,0,bow)
    return (np.multiply(tf,idf))
Vs_tfidf = tf_idf(Vs)
print(Vs_tfidf.shape)

(103, 1635)


In [94]:
 #Implementing k-Means Clustering
true_k = 2
km = KMeans(n_clusters=true_k, max_iter=100)
t0 = time()
km.fit(Vs_tfidf)
print("done in %0.3fs" % (time() - t0))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


done in 0.219s


In [98]:
centroids = km.cluster_centers_.argsort()[:, ::-1] ## Indices of largest centroids' entries in descending order
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in centroids[i, :1]: #making clusters basis one word (foodpoisoning)
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: foodpoisoning
Cluster 1: ketchup
